Prototyping notebook for predicting stock volaitility, prices, etc using extra data from web trends, news, etc. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable


Download:
https://www.kaggle.com/datasets/borismarjanovic/price-volume-data-for-all-us-stocks-etfs

https://www.kaggle.com/datasets/aceofit/stockmarketdatafrom1996to2020

https://www.kaggle.com/datasets/footballjoe789/us-stock-dataset

https://www.kaggle.com/datasets/paulrohan2020/crspusstocksdatabasequarterly

Datasets:
We need a historical dataset of stock prices.
Secondly, we need a dataset of web trends for the same time period. 

On the other hand, once the model is trained, it needs to be able to aquire live data that looks the same as the training data.

Possible sources:
   - twitter
   - google trends

Possible models:
    - Train a model to process text into an encoding
    - Train a model to do stock price prediction
    - Combine the two models to predict stock price/volatility (not sure which is best) from text with greater accuracy

In [2]:
# Collect text web data

In [3]:
# Collect stock data

In [4]:
# 